In [2]:
! pip install xclim 

  Using cached xclim-0.24.0-py3-none-any.whl (272 kB)
  Using cached boltons-20.2.1-py2.py3-none-any.whl (170 kB)
  Using cached cftime-1.4.1-cp38-cp38-manylinux2014_x86_64.whl (322 kB)
  Attempting uninstall: cftime
    Found existing installation: cftime 1.3.0
    Uninstalling cftime-1.3.0:
      Successfully uninstalled cftime-1.3.0


In [3]:
%matplotlib inline
import xarray as xr
import numpy as np 
import os 
import pandas as pd 
import matplotlib.pyplot as plt 

import intake
import zarr

import gcsfs

from xclim.core.calendar import convert_calendar

In [2]:
import warnings
warnings.filterwarnings("ignore")

For bias correction, we will be using 1995-2014 (the CMIP6 reference period). 

In [4]:
years = np.arange(1995, 2015)

function for removing leap days and converting the calendar to `noleap`

In [5]:
def convert_to_noleap_calendar(ds, target='noleap'):
    ds_noleap = convert_calendar(ds, target=target)
    return ds_noleap 

In [6]:
import dask
import dask.array as da
import dask.distributed as dd
import rhg_compute_tools.kubernetes as rhgk

In [7]:
fs = gcsfs.GCSFileSystem(token='/opt/gcsfuse_tokens/impactlab-data.json')

In [8]:
client, cluster = rhgk.get_standard_cluster()
cluster.scale(10)

In [11]:
client

Client Scheduler: gateway://traefik-impactlab-hub-dask-gateway.impactlab-hub:80/impactlab-hub.38d36f64266b48c88e00c6917d99287e Dashboard: /services/dask-gateway/clusters/impactlab-hub.38d36f64266b48c88e00c6917d99287e/status,Cluster Workers: 8 Cores: 8 Memory: 96.64 GB


In [21]:
cluster.close()

In [12]:
def create_era5_zarr_store(variable, years): 
    
    # create list of filenames to load
    filenames = [os.path.join('/gcs/impactlab-data/climate/source_data/ERA-5/day/%s/v1.1' %variable, 
                          '%s_daily_%s-%s.nc' %(variable, year, year)) for year in years]
    
    # Load files for specified years, note that the `preprocess=convert_to_noleap_calendar` argument is 
    # removing leap days and updating the calendar on each yearly file of daily data as it's being concatenated
    ds_allyears = xr.open_mfdataset(filenames, 
                                preprocess=convert_to_noleap_calendar)
    store_filename = 'gs://impactlab-data/climate/source_data/ERA-5/downscaling/%s.1995-2014.0p25.zarr' %variable 
    store = fs.get_mapper(store_filename, check=False)
    
    attrsdt = {'method': 'Changed calendar to no-leap and removed leap days'}
    ds_allyears.attrs.update(attrsdt)
    
    if not fs.exists(store_filename): 
        # save as a zarr store for rechunking 
        ds_allyears.to_zarr(store, consolidated=True, mode="w")

In [13]:
era5_variables = ['tmax', 'tmin', 'precip_total']

create zarr stores for tmax, tmin and precip_total

In [14]:
for variable in era5_variables: 
    create_era5_zarr_store(variable, years)

tmax


/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/note

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 7300)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) object 1995-01-01 00:00:00 ... 2014-12-31 00:00:00
Data variables:
    tmax       (time, latitude, longitude) float32 dask.array<chunksize=(365, 721, 1440), meta=np.ndarray>
Attributes:
    author:   Meredith Fish
    contact:  meredith.fish@rutgers.edu
    project:  impactlab-rhg/climate/source-data/ERA-5
    source:   impactlab-rhg/climate/downscaled/ERA-5/hourly
    created:  2021-02-16
    method:   Changed calendar to no-leap and removed leap days
tmin


/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/note

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 7300)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) object 1995-01-01 00:00:00 ... 2014-12-31 00:00:00
Data variables:
    tmin       (time, latitude, longitude) float32 dask.array<chunksize=(365, 721, 1440), meta=np.ndarray>
Attributes:
    author:   Meredith Fish
    contact:  meredith.fish@rutgers.edu
    project:  impactlab-rhg/climate/source-data/ERA-5
    source:   impactlab-rhg/climate/downscaled/ERA-5/hourly
    created:  2021-02-16
    method:   Changed calendar to no-leap and removed leap days
precip_total


/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/note

<xarray.Dataset>
Dimensions:       (latitude: 721, longitude: 1440, time: 7300)
Coordinates:
  * time          (time) object 1995-01-01 00:00:00 ... 2014-12-31 00:00:00
  * latitude      (latitude) float32 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude     (longitude) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
Data variables:
    precip_total  (time, latitude, longitude) float32 dask.array<chunksize=(365, 721, 1440), meta=np.ndarray>
Attributes:
    version:  v1.1
    author:   Meredith Fish
    contact:  meredith.fish@rutgers.edu
    project:  historical-climate-data-diagnostics
    source:   yearly files of ERA-5 daily data created from /gcs/impactlab-da...
    created:  2021-03-02
    method:   Changed calendar to no-leap and removed leap days
    units:    m


check to be sure xarray can read one of the test zarr stores

In [19]:
store_filename = 'gs://impactlab-data/climate/source_data/ERA-5/downscaling/precip_total.1995-2014.0p25.zarr' #%variable 
store = fs.get_mapper(store_filename, check=False)
ds_testread = xr.open_zarr(store, consolidated=True)
#ds_testread = xr.open_zarr(store)

In [20]:
ds_testread

<xarray.Dataset>
Dimensions:       (latitude: 721, longitude: 1440, time: 7300)
Coordinates:
  * latitude      (latitude) float32 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude     (longitude) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time          (time) object 1995-01-01 00:00:00 ... 2014-12-31 00:00:00
Data variables:
    precip_total  (time, latitude, longitude) float32 dask.array<chunksize=(365, 721, 1440), meta=np.ndarray>
Attributes:
    author:   Meredith Fish
    contact:  meredith.fish@rutgers.edu
    created:  2021-03-02
    method:   Changed calendar to no-leap and removed leap days
    project:  historical-climate-data-diagnostics
    source:   yearly files of ERA-5 daily data created from /gcs/impactlab-da...
    units:    m
    version:  v1.1